<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/005-EVI-with-Overloaded-Operators-Sentinel-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Overloaded Operators to Compute the EVI (Sentinel-2)

## Let's start!

If required, please uncomment:

In [1]:
!pip install eemont
!pip install geemap

  Created wheel for eemont: filename=eemont-0.1.7-cp36-none-any.whl size=16387 sha256=26a00645ba38879eb91b9715cc74be4c7e31bb91ba2642d7338c1d65ce627664
  Stored in directory: /root/.cache/pip/wheels/ca/ba/9a/3fc1205f53a16e573daef50a4db1d759940ad173acca050589
Successfully built eemont
     |████████████████████████████████| 399kB 5.4MB/s 
     |████████████████████████████████| 3.3MB 6.3MB/s 
     |████████████████████████████████| 5.1MB 47.5MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 143kB 48.3MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 2.5MB 47.1MB/s 
     |████████████████████████████████| 225kB 25.2MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
     |████████████████████████████████| 122kB 37.4MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 552kB 45.7

Import the required packges.

In [2]:
import ee, eemont, datetime, geemap

Authenticate and Initialize Earth Engine and geemap.

In [3]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=urkezwJQL787C5hPP76w__YllUGiVDWHTjwM2-a4fZg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5DzLruP--5tshGiSnSpfD3_MmJ3GkLCCYIdH9ho78-5bmH0g7V-lw

Successfully saved authorization token.


Point of interest.

In [5]:
point = ee.Geometry.Point([-76.0269,2.92846])

Get, filter, mask clouds and scale the image collection.

In [6]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scale()
      .index('EVI')) # Let's compute the EVI using the index() method for comparison

Let's select the required bands for EVI:

In [7]:
N = S2.select('B8')
R = S2.select('B4')
B = S2.select('B2')

## Overloaded Operators

`eemont` has overloaded the binary operators, rich comparisons and unary operators in the following list for the `ee.Image` class:

(+, -, \*\, /, //, %, \**\, <<, >>, &, |, <, <=, ==, !=, >, >=, -, ~)

Therefore, you can now use them for image operations!

In [8]:
EVI = 2.5 * (N - R) / (N + 6.0 * R - 7.5 * B + 1.0)

## Visualization

Let's define the EVI visualization parameters:

In [10]:
visEVI = {
    'min':0,
    'max':1,
    'palette':[
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000',
        '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301'
    ]    
}

And also the RGB visualization parameters:

In [11]:
visRGB = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']
}

Use `geemap` to display results:

In [12]:
Map.addLayer(S2,visRGB,'RGB')
Map.addLayer(EVI,visEVI,'EVI With Overloaded Operators')
Map.addLayer(S2.select('EVI'),visEVI,'EVI With index() Method')
Map.add_colorbar(visEVI['palette'], caption = 'EVI')
Map.centerObject(point)
Map.addLayerControl()
Map